# 🔤 MX-Font 한글 폰트 생성기

**43~256자 손글씨 샘플로 11,172자 한글 폰트 생성**

네이버 클로바 AI의 MX-Font 기술 기반

---

## 사용 방법

1. **런타임 설정**: `런타임 > 런타임 유형 변경 > GPU` 선택
2. **셀 순서대로 실행**: 각 셀을 위에서 아래로 실행
3. **손글씨 이미지 업로드**: 템플릿에 맞춰 작성한 손글씨 이미지 업로드
4. **폰트 생성**: 자동으로 11,172자 폰트 생성
5. **다운로드**: 생성된 TTF 파일 다운로드

## 1️⃣ 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

import torch
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# MX-Font 저장소 클론
!git clone https://github.com/clovaai/fewshot-font-generation.git
%cd fewshot-font-generation

# 필수 패키지 설치
!pip install -q sconf munch fonttools pillow opencv-python tqdm

In [ ]:
# 추가 의존성 설치
!pip install -q lmdb fire

## 2️⃣ 샘플 글자 세트 정의

In [ ]:
# 샘플 글자 세트 정의

# 8자 세트 (빠른 테스트용)
CHARS_8 = list("가나다라마바사아")

# 28자 세트 (기본 품질)
CHARS_28 = list("갈같강개걔거겨계고과괴구궈귀그긔기깨꺼꼬꾸끼냐녀노뇨누뉴")

# 43자 세트 (좋은 품질)
CHARS_43 = list("각간감갑객건걸검겁게격견결경곡곤골공관광국군굴궁권균극근글금급긍긴길김깊꽃꿈끝낙난날남")

# 256자 세트 (최고 품질)
CHARS_256_STR = """가각간갈감갑강개객갱거건걸검게겨격견결경계고곡곤골공과관광괴구국
군굴궁권귀규균그극근글금급긍기긴길김깊까깨꺼께꼬꽃꾸꿈끄끌끝나낙
난날남납낭내냉너널네녀년념녕노녹논놀농뇌누눈눌느늘능니닉닌닐님다
단달담답당대댁더덕던덜덤덩데도독돈돌동두둔둘둥드득든들등디따땅때
떠또뚜뜨뜻라락란랄람랍랑래랭러럭런럴렁레려력련렬렴령례로록론롤롱
뢰료루룩룬룰룸류륙륜률륭르륵른를름릉리릭린릴림립마막만말망매맥맹
머먹멀멍메며면멸명모목몰몽묘무묵문물뭇므미민밀밍바박반발밤밥방배
백번벌범법벽변별병보복본볼봉부북분불붕비빈빌빙빠빼뻐뽀뿌쁘사삭산
살삼삽상새색생서석선설섬섭성세소속손솔송쇄쇠수숙순술숭쉬스슬습승
시식신실심십싱싸쌍써쏘쑤쓰씨아악안알암압앙애액앵야약양어억언얼엄
업엉에여역연열염엽영예오옥온올옹와완왕왜외요욕용우욱운울웅원월위
유육윤율융으은을음읍응의이익인일임입잉자작잔잘잠잡장재쟁저적전절
점접정제조족존졸종좌죄주죽준줄중즉즐즘증지직진질짐집징차착찬찰참
창채책처척천철첨첩청체초촉촌총최추축춘출충취측층치칙친칠침칭쾌타
탁탄탈탐탑탕태택터털토톤통퇴투퉁트특틈티파판팔패팽퍼펴편평폐포폭
표푸품풍프피픽필핍하학한할함합항해핵행향허헌험혁현혈협형혜호혹혼
홀홍화확환활황회획횡효후훈훌훙휘휴흉흐흑흔흘흥희히힘"""
CHARS_256 = [c for c in CHARS_256_STR if c.strip()]

print(f"8자 세트: {len(CHARS_8)}자")
print(f"28자 세트: {len(CHARS_28)}자")
print(f"43자 세트: {len(CHARS_43)}자")
print(f"256자 세트: {len(CHARS_256)}자")

## 3️⃣ 손글씨 이미지 업로드

In [ ]:
from google.colab import files
import os

# 업로드 디렉토리 생성
os.makedirs('user_handwriting', exist_ok=True)

print("손글씨 이미지를 업로드하세요.")
print("- 각 글자별 개별 이미지 (예: 가.png, 나.png, ...)")
print("- 또는 템플릿 스캔 이미지")

uploaded = files.upload()

for filename in uploaded.keys():
    # 업로드된 파일을 user_handwriting 폴더로 이동
    os.rename(filename, f'user_handwriting/{filename}')
    print(f"업로드 완료: {filename}")

## 4️⃣ 이미지 전처리

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

def preprocess_template_image(image_path, chars, output_dir, grid_cols=8):
    """
    템플릿 이미지에서 개별 글자 추출
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # 이미지 로드
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 이진화
    _, binary = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    
    # 컨투어 찾기
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 바운딩 박스로 정렬
    boxes = [cv2.boundingRect(c) for c in contours]
    boxes = sorted(boxes, key=lambda b: (b[1] // 50, b[0]))  # y, x 순으로 정렬
    
    # 각 글자 추출
    for i, (x, y, w, h) in enumerate(boxes[:len(chars)]):
        char_img = gray[y:y+h, x:x+w]
        
        # 128x128로 리사이즈
        char_img = cv2.resize(char_img, (128, 128))
        
        # 저장
        char = chars[i]
        output_path = os.path.join(output_dir, f"{ord(char):04X}.png")
        cv2.imwrite(output_path, char_img)
    
    print(f"{len(boxes[:len(chars)])}개 글자 추출 완료")


def preprocess_individual_images(input_dir, output_dir):
    """
    개별 글자 이미지 전처리
    파일명이 글자인 경우 (예: 가.png)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for filename in os.listdir(input_dir):
        if not filename.endswith(('.png', '.jpg', '.jpeg')):
            continue
        
        # 파일명에서 글자 추출
        char = os.path.splitext(filename)[0]
        if len(char) != 1:
            continue
        
        # 이미지 로드 및 전처리
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 128x128로 리사이즈
        img = cv2.resize(img, (128, 128))
        
        # 유니코드 코드포인트로 저장
        output_path = os.path.join(output_dir, f"{ord(char):04X}.png")
        cv2.imwrite(output_path, img)
    
    print(f"전처리 완료: {output_dir}")

In [ ]:
# 전처리 실행
# 개별 이미지인 경우
preprocess_individual_images('user_handwriting', 'processed_chars')

# 처리된 파일 확인
processed_files = os.listdir('processed_chars')
print(f"처리된 글자 수: {len(processed_files)}")

## 5️⃣ MX-Font 모델 설정

In [ ]:
# 설정 파일 확인
!ls -la cfgs/
!cat cfgs/MX-Font/default.yaml

In [ ]:
# 한글 데이터 설정 확인
!ls -la data/kor/
!head -20 data/kor/train_chars.json

## 6️⃣ 모델 학습 (선택사항)

사전 학습된 모델이 없는 경우 직접 학습이 필요합니다.
학습에는 여러 TTF 폰트 파일이 필요합니다.

In [ ]:
# 학습 실행 (시간이 오래 걸림)
# !python train.py cfgs/MX-Font/train.yaml

## 7️⃣ 폰트 생성

In [ ]:
# 폰트 생성 (사전 학습 모델 필요)
# !python evaluator.py cfgs/MX-Font/eval.yaml --weight <checkpoint_path>

## 8️⃣ 이미지를 TTF로 변환

In [ ]:
!pip install -q fontforge potrace

# fontforge 설치 (시스템 패키지)
!apt-get install -y fontforge python3-fontforge potrace

In [ ]:
import subprocess
import os

def images_to_ttf(image_dir, output_ttf, font_name="MyHandwriting"):
    """
    글자 이미지들을 TTF 폰트로 변환
    """
    # FontForge 스크립트 생성
    script = f'''
import fontforge
import os

font = fontforge.font()
font.familyname = "{font_name}"
font.fullname = "{font_name}"
font.fontname = "{font_name}"
font.encoding = "UnicodeFull"

image_dir = "{image_dir}"

for filename in os.listdir(image_dir):
    if not filename.endswith(".png"):
        continue
    
    # 파일명에서 유니코드 코드포인트 추출
    codepoint = int(os.path.splitext(filename)[0], 16)
    
    # 글리프 생성
    glyph = font.createChar(codepoint)
    
    # 이미지 임포트
    image_path = os.path.join(image_dir, filename)
    glyph.importOutlines(image_path)
    glyph.width = 1000

font.generate("{output_ttf}")
print("폰트 생성 완료: {output_ttf}")
'''
    
    # 스크립트 저장 및 실행
    with open('create_font.py', 'w') as f:
        f.write(script)
    
    subprocess.run(['fontforge', '-script', 'create_font.py'])

# 실행
# images_to_ttf('generated_chars', 'MyHandwriting.ttf')

## 9️⃣ 결과 다운로드

In [ ]:
from google.colab import files

# 생성된 폰트 다운로드
# files.download('MyHandwriting.ttf')

---

## 📚 참고 자료

- [MX-Font GitHub](https://github.com/clovaai/fewshot-font-generation)
- [DM-Font 논문](https://arxiv.org/abs/2005.10510)
- [LF-Font 논문](https://arxiv.org/abs/2009.11042)

## ⚠️ 주의사항

1. GPU 런타임 필수 (무료 T4 또는 유료 A100)
2. 학습에는 여러 TTF 폰트 파일이 필요
3. 상업적 사용 시 라이선스 확인 필요